In [ ]:
import os, io
import numpy as np
import datetime

import pandas as pd
pd.set_option("display.precision", 3)

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.ticker import FormatStrFormatter

import warnings
warnings.filterwarnings("ignore")
from utilities import *

In [ ]:
def main(date):
    path_date = os.path.join('data', date)
    path_feces = os.path.join(path_date, 'feces.xlsx')
    path_urine = os.path.join(path_date, 'urine.xlsx')
    path_flowmeter = os.path.join(path_date, 'flowmeter.xlsx')
    df_feces = read_data(path_feces, date) # read feces file
    df_feces = df_feces[['weight']]
    df_feces.columns = ['feces']
    df_urine = read_data(path_urine, date) # read feces file
    df_urine = df_urine[['weight']]
    df_urine.columns = ['urine']
    df_flowmeter = read_data(path_flowmeter, date, is_interpolate=False) # read feces file
    df_flowmeter.index = df_flowmeter.index + pd.Timedelta(hours=8, minutes=54, seconds=34)
    df_flowmeter = df_flowmeter[['STALL1']]
    df_flowmeter.columns = ['flow']
    df_flowmeter.loc[:, 'flow'] = df_flowmeter.loc[:, 'flow']/60
    df = pd.concat([df_feces, df_urine, df_flowmeter], axis=1)
    df = df.reset_index()
    df ['feces_derivative'] = df['feces'].diff(periods=10) / df['date_time'].diff(periods=10).dt.total_seconds()
    df ['urine_derivative'] = df['urine'].diff(periods=10) / df['date_time'].diff(periods=10).dt.total_seconds()
    start_indexes, end_indexes = detect_event(df.loc[:], path_date)
    df_stat = get_stat(df, start_indexes, end_indexes)
    df_stat.to_csv(os.path.join(path_date, 'stat.csv'), index=False)
    for file in os.listdir(path_date):
        if file.endswith('.png'):
            os.remove(os.path.join(path_date, file)) 
    for i, (start_index, end_index) in enumerate(zip(start_indexes, end_indexes)):  
        df_event, _, _, _, _ = process_event(df, start_index, end_index)
        fig = plot_event(df_event, [0], [df_event.index[-1]])
        plt.show()
        fig.savefig(os.path.join(path_date, 'event_' + str(i+1) + '.png'))
        fig = plot_derivative(df_event, [0], [df_event.index[-1]])
        plt.show()
        fig.savefig(os.path.join(path_date, 'derivative_' + str(i+1) + '.png'))  
    return df_stat

In [ ]:
for date in ['20201124', '20201125', '20201210', '20201211', 
             '20201212', '20201214', '20201215', '20201221', '20201222', 
             '20201223', '20201224', '20201228', '20201229', '20201230']:
    main(date)